In [4]:
#python manage.py shell_plus --notebook
import os, sys
os.chdir(os.path.dirname(os.path.realpath('.')))
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [2]:
from users.common import *
from users.models import *

In [ ]:
from users.common import *

class get_query(object):
    def queryset(self):
        module=self.model
#         if not self.request.user.is_authenticated:
#             print('#########user not logged in#########')
#             return module.objects.none()
        self.section=ProfileRole.objects.get(user__user__username=self.username).depertment
        print(self.username,self.model_id)
        try:
            if self.section.id:
                query_set=Scope.objects.get(module__id=self.model_id).query_set
                self.section_id=self.section.id
                print('self.section_id',self.section_id)
                query_set='self.query_set='+query_set
                print(query_set)
                exec(query_set)
                return self.query_set
        except Exception as e:
            print(e)
            try:
                if self.section == None:
                    print('#########user has all perm#########')
                    return self.model.objects.all()
            except Exception as e:
                print(e)
                print('#########user not permitted#########')
                return self.model.objects.none()
            
#     def __init__(self,request,model):
#         self.model=model
#         self.request=request
#         self.model_name=fundamental().get_class_name(model)
#         self.model_id=fundamental().get_model_id(model)
#         self.method=request.method
#         self.models=fundamental().get_all_models()
#         if request.user.is_authenticated:
#             self.user=request.user
#             self.username=request.user.username
#             self.role_id=fundamental().get_role_id(self.username)
#             self.role_name=fundamental().get_role_name(self.username)
#             self.section=ProfileRole.objects.get(user__user__username=self.username).depertment
            
    def __init__(self,model,method,username):
        self.model=model
        self.model_name=fundamental().get_class_name(model)
        self.model_id=fundamental().get_model_id(model)
        self.method=method
        self.models=fundamental().get_all_models()
        self.username=username
        self.role_id=fundamental().get_role_id(self.username)
        self.role_name=fundamental().get_role_name(self.username)
        self.section=ProfileRole.objects.get(user__user__username=self.username).depertment


In [ ]:
get_query(Test,'GET','mojo').queryset()

In [107]:
a=1
stra='query_set=Test.objects.filter(section__id=a)'
exec(stra)
query_set

<QuerySet [<Test: Milk->Protein>, <Test: Milk->Carbs>]>

In [ ]:
country_name='India'
avg_age=26

In [ ]:
'name=country_name'
'age=avg_age'
'defence_budget=budget'

In [ ]:
'model.objects.filter('+'name=country_name'+')'

In [5]:
from users.filter import *

In [6]:
Section.objects.filter(id=4)

<QuerySet [<Section: Fried Snacks>]>